In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as ssp

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics, model_selection
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm
from lightgbm.sklearn import LGBMClassifier
import os 
import re
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
work_train = {} 
work_test = {}
pre_process=["d2v.npz","tfidf.npz","w2v.npz"]
path="checkpoints_databases/"
for f in pre_process:
    work_train[re.sub("\.npz","",f)] = ssp.load_npz(path+"nw_working_train_"+f)
    work_test[re.sub("\.npz","",f)] = ssp.load_npz(path+"nw_working_test_"+f)

In [3]:
for a in work_train:
    print(a)

d2v
tfidf
w2v


In [4]:
y=pd.read_csv("..//bases/training_variants").Class.values - 1

In [5]:
train = pd.read_csv('..//bases/training_variants')
test = pd.read_csv('..//bases/test_variants')
ID_train=train.ID
ID_test=test.ID
del train,test

# GRID SEARCH PHASE ALL ALGOS

In [ ]:
clf_xgb=XGBClassifier(max_depth=3, objective="multi:softprob",seed=26)
param_test= {
    "max_depth" : [3,5,7],
    "min_child_weight" : [1,3],
    "n_estimators" : [100,200],
    "subsample":[0.8,1],
    "colsample_bytree":[0.8,1]
}
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_xgb, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=3,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)

In [ ]:
clf_lgbm=LGBMClassifier(seed=26)
param_test= {
    'n_estimators': [8,24,48],
    'num_leaves': [6,12,16,22],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'colsample_bytree' : [0.7 0.8],
    'subsample' : [0.7,0.8]
    }
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_lgbm, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=3,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)

In [ ]:
clf_log=LogisticRegression()
param_test= {
    "C" : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "penalty" : ["l1","l2"]
}
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_log, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=3,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)

In [ ]:
clf_ada=AdaBoostClassifier(n_estimators=100, learning_rate=1.0, algorithm="SAMME.R", random_state=26)
param_test={
    "n_estimators":[50,100],
    "learning_rate":[0.3,0.5,0.9,1]
}
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_ada, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=3,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)

In [ ]:
clf_dt=RandomForestClassifier()
param_test={
    "max_depth":[5,7,10,15],
    "n_estimators":[100,200,300]
}
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_dt, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=3,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)

# TRAINING PHASE ALL ALGOS 

In [6]:
def model_gen(X,X_test,y,classifier,file,five_fold_predict=True):
    #if not os.path.exists("scores/"+file):
    #   os.makedirs("scores/"+file)
    kf = model_selection.StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    if five_fold_predict:
        fold = 0
        y_test=0
        for train_index, test_index in kf.split(X, y):
        
            fold += 1

            X_train, X_valid    = X[train_index],   X[test_index]
            y_train, y_valid    = y[train_index],   y[test_index]

            print("Fold", fold, X_train.shape, X_valid.shape)

            clf=classifier
            clf.fit(X_train,y_train)
            p_test = clf.predict_proba(X_test)
            y_test += p_test/5

    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("nw_scores/nw_stack_test/nw_{}.csv".format(file),index=False)
    
    print("cross_val sur train ") #peut etre que to array est exclusivement pour les xgb
    
    if os.path.isfile("nw_scores/nw_stack_train/nw_{}.csv".format(file)):
        print("not necessary, already done")
    else:
        y_pred=cross_val_predict(estimator=clf,X=X,y=y,cv=kf,method="predict_proba")
        subm1 = pd.DataFrame(y_pred, columns=classes)
        subm1['ID'] = ID_train
        subm1.to_csv("nw_scores/nw_stack_train/nw_{}.csv".format(file),index=False)


In [ ]:
print("xgboost here")
for clf,name in zip(dic_xgb.keys(),work_train.keys()):
    model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_xgb[clf],file=clf)

xgboost here
Fold 1 (2653, 629) (668, 629)
Fold 2 (2654, 629) (667, 629)
Fold 3 (2657, 629) (664, 629)
Fold 4 (2659, 629) (662, 629)
Fold 5 (2661, 629) (660, 629)
cross_val sur train 
Fold 1 (2653, 10279) (668, 10279)
Fold 2 (2654, 10279) (667, 10279)
Fold 3 (2657, 10279) (664, 10279)

In [11]:
dic_xgb={"xgb_d2v":XGBClassifier(max_depth=3,objective="multi:softprob",seed=26),
        "xgb_tfidf":XGBClassifier(max_depth=3,objective="multi:softprob",seed=26),
        "xgb_w2v":XGBClassifier(max_depth=3,objective="multi:softprob",seed=26)}
dic_lgbm={"lgbm_d2v":LGBMClassifier(seed=26),
        "lgbm_tfidf":LGBMClassifier(seed=26),
        "lgbm_w2v":LGBMClassifier(seed=26)}
dic_lr={"lr_d2v":LogisticRegression(),
        "lr_tfidf":LogisticRegression(),
        "lr_w2v":LogisticRegression()}
dic_ada={"ada_d2v":AdaBoostClassifier(n_estimators=100, learning_rate=1.0, algorithm="SAMME.R", random_state=26),
        "ada_tfidf":AdaBoostClassifier(n_estimators=100, learning_rate=1.0, algorithm="SAMME.R", random_state=26),
        "ada_w2v":AdaBoostClassifier(n_estimators=100, learning_rate=1.0, algorithm="SAMME.R", random_state=26)}
dic_rf={"rf_d2v":RandomForestClassifier(n_estimators=200,max_depth=10,random_state=26),
        "rf_tfidf":RandomForestClassifier(n_estimators=200,max_depth=10,random_state=26),
        "rf_w2v":RandomForestClassifier(n_estimators=200,max_depth=10,random_state=26)}

print("xgboost here")
for clf,name in zip(dic_xgb.keys(),work_train.keys()):
    model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_xgb[clf],file=clf)
print("lgbm here")
for clf,name in zip(dic_lgbm.keys(),work_train.keys()):
    model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_lgbm[clf],file=clf)
print("logreg here")
for clf,name in zip(dic_lr.keys(),work_train.keys()):
    model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_lr[clf],file=clf)
print("adaboost here")
for clf,name in zip(dic_ada.keys(),work_train.keys()):
    model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_ada[clf],file=clf)
print("random forest here")
for clf,name in zip(dic_rf.keys(),work_train.keys()):
        model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_rf[clf],file=clf)


Fold 1 (2653, 629) (668, 629)
Fold 2 (2654, 629) (667, 629)
Fold 3 (2657, 629) (664, 629)
Fold 4 (2659, 629) (662, 629)
Fold 5 (2661, 629) (660, 629)
cross_val sur train 
Fold 1 (2653, 10279) (668, 10279)
Fold 2 (2654, 10279) (667, 10279)
Fold 3 (2657, 10279) (664, 10279)
Fold 4 (2659, 10279) (662, 10279)
Fold 5 (2661, 10279) (660, 10279)
cross_val sur train 
not necessary, already done
Fold 1 (2653, 379) (668, 379)
Fold 2 (2654, 379) (667, 379)
Fold 3 (2657, 379) (664, 379)
Fold 4 (2659, 379) (662, 379)
Fold 5 (2661, 379) (660, 379)
cross_val sur train 
not necessary, already done
Fold 1 (2653, 629) (668, 629)
Fold 2 (2654, 629) (667, 629)
Fold 3 (2657, 629) (664, 629)
Fold 4 (2659, 629) (662, 629)
Fold 5 (2661, 629) (660, 629)
cross_val sur train 
Fold 1 (2653, 10279) (668, 10279)
Fold 2 (2654, 10279) (667, 10279)
Fold 3 (2657, 10279) (664, 10279)
Fold 4 (2659, 10279) (662, 10279)
Fold 5 (2661, 10279) (660, 10279)
cross_val sur train 
not necessary, already done
Fold 1 (2653, 379) (6

KeyboardInterrupt: 